In [38]:
import pandas as pd
import numpy as np
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split

In [39]:
from google.colab import files
uploaded = files.upload()

In [40]:
cf_data = pd.read_csv("cf_data.csv")
cf_labels = pd.read_csv("cf_labels.csv")
cf_labels['Should_Change_Enzyme'] = cf_labels['Should_Change_Enzyme'].astype(np.float64)
cf_data['Ate_Allergic'] = cf_data['Ate_Allergic'].astype(np.float64)
print(cf_data.dtypes)
print(cf_labels.dtypes)

Enzyme_Difference    float64
Oz_Of_Water          float64
Exercise_Length      float64
Stress_Level         float64
Ate_Allergic         float64
Stool_Rating         float64
dtype: object
Should_Change_Enzyme    float64
dtype: object


In [0]:
def get_normalization_parameters(traindf, features):
    """Get the normalization parameters (E.g., mean, std) for traindf for 
    features. We will use these parameters for training, eval, and serving."""

    def _z_score_params(column):
        mean = traindf[column].mean()
        std = traindf[column].std()
        return {'mean': mean, 'std': std}

    normalization_parameters = {}
    for column in features:
        normalization_parameters[column] = _z_score_params(column)
    return normalization_parameters

NUMERIC_FEATURES = ['Enzyme_Difference', 'Oz_Of_Water', 'Exercise_Length',
                    'Stress_Level', 'Ate_Allergic', 'Stool_Rating']
normalization_parameters = get_normalization_parameters(cf_data, NUMERIC_FEATURES)

In [0]:
def numeric_column_normalized(column_name, normalizer_fn):
    return tf.feature_column.numeric_column(column_name, normalizer_fn=normalizer_fn)

def make_zscaler(mean, std):
    def zscaler(col):
        return (col - mean)/std
    return zscaler

# Define your feature columns
def create_feature_cols(features, use_normalization):
    """Create feature columns using tf.feature_column. 
    
    This function will get executed during training, evaluation, and serving."""
    def normalize_column(col):  # Use mean, std defined below.
        return (col - mean)/std
    normalized_feature_columns = []
    for column_name in features:
        normalizer_fn = None
        if use_normalization:
            column_params = normalization_parameters[column_name]
            mean = column_params['mean']
            std = column_params['std']
            normalizer_fn = make_zscaler(mean, std)
        normalized_feature_columns.append(numeric_column_normalized(column_name,
                                                                     normalizer_fn))
    return normalized_feature_columns

In [0]:
NUMERIC_FEATURES = ['Enzyme_Difference', 'Oz_Of_Water', 'Exercise_Length',
                    'Stress_Level', 'Ate_Allergic', 'Stool_Rating']
feature_columns = create_feature_cols(NUMERIC_FEATURES, use_normalization=True)

In [46]:
X_train, X_test, y_train, y_test = train_test_split(cf_data,cf_labels["Should_Change_Enzyme"],test_size=0.25, random_state=101)
print(y_train)
inputFunction = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=10,num_epochs=1000,shuffle=True)

7     0.0
34    0.0
22    1.0
10    1.0
3     0.0
33    0.0
28    1.0
24    1.0
2     0.0
35    0.0
15    0.0
19    0.0
37    1.0
29    0.0
12    0.0
5     1.0
14    0.0
0     0.0
21    0.0
39    0.0
4     0.0
8     1.0
13    1.0
9     0.0
38    0.0
23    0.0
6     1.0
17    0.0
11    0.0
31    1.0
Name: Should_Change_Enzyme, dtype: float64


In [47]:
dnnClassifierModel = tf.estimator.DNNClassifier(hidden_units=[512, 256, 128], feature_columns=feature_columns, n_classes=2, activation_fn=tf.nn.tanh,optimizer=lambda: tf.train.AdamOptimizer(learning_rate=tf.train.exponential_decay(learning_rate=0.001, global_step=tf.train.get_global_step(), decay_steps=1000, decay_rate=0.96)))
dnnClassifierModel.train(input_fn=inputFunction)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpbslgcwo2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f704c806630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Vari

In [48]:
evaluateInputFunction = tf.estimator.inputs.pandas_input_fn(
 x=X_test,
 y=y_test,
 batch_size=10,
 num_epochs=1,
 shuffle=False)
dnnClassifierModel.evaluate(evaluateInputFunction)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-01-27T01:24:45Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbslgcwo2/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-01-27-01:24:46
INFO:tensorflow:Saving dict for global step 3000: accuracy = 0.6, accuracy_baseline = 0.8, auc = 0.81249964, auc_precision_recall = 0.96249986, average_loss = 3.4540105, global_step = 3000, label/mean = 0.8, loss = 34.540104, precision = 1.0, prediction/mean = 0.42234778, recall = 0.5
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: /tmp/tmpbslgcwo2/model.ckpt-3000


{'accuracy': 0.6,
 'accuracy_baseline': 0.8,
 'auc': 0.81249964,
 'auc_precision_recall': 0.96249986,
 'average_loss': 3.4540105,
 'global_step': 3000,
 'label/mean': 0.8,
 'loss': 34.540104,
 'precision': 1.0,
 'prediction/mean': 0.42234778,
 'recall': 0.5}

In [0]:
def predict_input_fn(data_file):
        global CSV_COLUMNS
        CSV_COLUMNS = CSV_COLUMNS[:-1]
        df_data = pd.read_csv(
            tf.gfile.Open(data_file),
            names=CSV_COLUMNS,
            skipinitialspace=True,
            engine='python',
            skiprows=1
        )

        # remove NaN elements
        df_data = df_data.dropna(how='any', axis=0)

        return tf.estimator.inputs.pandas_input_fn(
            x=df_data,
            num_epochs=1,
           shuffle=False
        )

In [0]:
def getAnswer (enzDif, water, exLen, stress, ateAll, stool):
  d = {'Enzyme_Difference': [enzDif], 'Oz_Of_Water': [water], 'Exercise_Length': [exLen], 'Stress_Level': [stress], 'Ate_Allergic': [ateAll], 'Stool_Rating': [stool]}
  df = pd.DataFrame(data=d)
  #print(df)
  inputFuncPredict = tf.estimator.inputs.pandas_input_fn(x=df, num_epochs=1, shuffle=False)
  results = dnnClassifierModel.predict(input_fn=inputFuncPredict)

  for result in results:
    formatted = format(result)
    #print(formatted)
    split = formatted.split(", ")
    answer = split[8].split("'")[3]
    return answer

In [0]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

cred = credentials.Certificate("serviceAccountKey.json")



In [52]:
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://cf-tracker-pro.firebaseio.com'
})

ValueError: ignored

In [64]:
ref = db.reference('/endDayTests')

tests = ref.get(user)[0]

firstUser = str(tests).split("}")[0]

username = firstUser.split(": ")[0].replace("\'", "")
username = username.replace("{", "")
#print(username)

#ateAll = firstUser.split(": ")[2]
#if ateAll.find(", ") != -1:
#    ateAll = ateAll.split(", ")[0]
#print(ateAll)
enzDif = firstUser.split(": ")[2]
if enzDif.find(", ") != -1:
    enzDif = enzDif.split(", ")[0]
#print(enzDif)
exLen = firstUser.split(": ")[3]
if exLen.find(", ") != -1:
    exLen = exLen.split(", ")[0]
#print(exLen)
ozWat = firstUser.split(": ")[4]
if ozWat.find(", ") != -1:
    ozWat = ozWat.split(", ")[0]
#print(ozWat)
stool = firstUser.split(": ")[5]
if stool.find(", ") != -1:
    stool = stool.split(", ")[0]
#print(stool)
#stress = firstUser.split(": ")[7]
#if stress.find(", ") != -1:
#    stress = stress.split(", ")[0]
#print(stress)

ans = getAnswer(float(enzDif), float(ozWat), float(exLen), float(0), float(0), float(stool))

ref.update({
    username: {
        'Change_Enzyme': ans
    }
})

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpbslgcwo2/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
